In [12]:
import numpy as np
import pandas as pd 


In [13]:
anime = pd.read_csv(r"D:\self\anime recomendations system\archive\anime.csv")
rating= pd.read_csv(r"D:\self\anime recomendations system\archive\rating.csv")

In [14]:
anime.head()
print(anime.shape)
print(rating.shape)

(12294, 7)
(7813737, 3)


In [15]:
anime.isnull().sum()


anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64

In [16]:
anime.dropna(inplace=True)


In [17]:
anime.duplicated().sum()

0

In [18]:
rating.duplicated().sum()

1

In [19]:
# Remove duplicates
rating = rating.drop_duplicates()


In [20]:
anime.shape

(12017, 7)


popularity based recommender System
display top 100 both with highest rating with more than 200 users rating

In [21]:
anime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [22]:
rating.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [23]:
rating_with_name=rating.merge(anime,on='anime_id')

In [24]:
rating_with_name.head()

,user_id,anime_id,rating_x,name,genre,type,episodes,rating_y,members
0,1,20,-1,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297
1,1,24,-1,School Rumble,"Comedy, Romance, School, Shounen",TV,26,8.06,178553
2,1,79,-1,Shuffle!,"Comedy, Drama, Ecchi, Fantasy, Harem, Magic, R...",TV,24,7.31,158772
3,1,226,-1,Elfen Lied,"Action, Drama, Horror, Psychological, Romance,...",TV,13,7.85,623511
4,1,241,-1,Girls Bravo: First Season,"Comedy, Ecchi, Fantasy, Harem, Romance, School",TV,11,6.69,84395


In [25]:
num_rating_df=rating_with_name.groupby('name').count()['rating_y'].reset_index()
num_rating_df.rename(columns={'rating_y':'num_rating'},inplace=True)
num_rating_df

,name,num_rating
0,&quot;0&quot;,26
1,&quot;Aesop&quot; no Ohanashi yori: Ushi to Ka...,2
2,&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hat...,782
3,&quot;Bungaku Shoujo&quot; Memoire,809
4,&quot;Bungaku Shoujo&quot; Movie,1535
...,...,...
11156,xxxHOLiC Kei,3413
11157,xxxHOLiC Movie: Manatsu no Yoru no Yume,2365
11158,xxxHOLiC Rou,1513
11159,xxxHOLiC Shunmuki,1974


In [26]:

print(rating_with_name['rating_y'].dtype)


float64


In [27]:
print(rating_with_name['rating_y'].isnull().sum())


0


In [28]:
# Fill NaN values with 0
rating_with_name['rating_y'].fillna(0, inplace=True)


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_17792\3203038683.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  rating_with_name['rating_y'].fillna(0, inplace=True)


In [29]:
print(rating_with_name['name'].dtype)  # Should be 'object' for strings
print(rating_with_name['name'].isnull().sum())  # Check for missing values


object
0


In [30]:
rating_with_name['rating_y'] = pd.to_numeric(rating_with_name['rating_y'], errors='coerce')


In [31]:
print(rating_with_name.dtypes)


user_id       int64
anime_id      int64
rating_x      int64
name         object
genre        object
type         object
episodes     object
rating_y    float64
members       int64
dtype: object


In [32]:
# Perform Groupby and calculate the mean for 'rating_y', ignoring non-numeric columns
avg_rating_df = rating_with_name.groupby('name').mean(numeric_only=True)['rating_y'].reset_index()

# Rename 'rating_y' to 'avg_rating'
avg_rating_df.rename(columns={'rating_y': 'avg_rating'}, inplace=True)

# Display the first 5 rows of the result
print(avg_rating_df)


                                                    name  avg_rating
0                                          &quot;0&quot;        5.06
1      &quot;Aesop&quot; no Ohanashi yori: Ushi to Ka...        5.00
2      &quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hat...        7.06
3                     &quot;Bungaku Shoujo&quot; Memoire        7.54
4                       &quot;Bungaku Shoujo&quot; Movie        7.63
...                                                  ...         ...
11156                                       xxxHOLiC Kei        8.34
11157            xxxHOLiC Movie: Manatsu no Yoru no Yume        8.04
11158                                       xxxHOLiC Rou        8.32
11159                                  xxxHOLiC Shunmuki        8.17
11160                                                  ◯        4.22

[11161 rows x 2 columns]


In [33]:
popular_df=num_rating_df.merge(avg_rating_df,on='name')
popular_df

,name,num_rating,avg_rating
0,&quot;0&quot;,26,5.06
1,&quot;Aesop&quot; no Ohanashi yori: Ushi to Ka...,2,5.00
2,&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hat...,782,7.06
3,&quot;Bungaku Shoujo&quot; Memoire,809,7.54
4,&quot;Bungaku Shoujo&quot; Movie,1535,7.63
...,...,...,...
11156,xxxHOLiC Kei,3413,8.34
11157,xxxHOLiC Movie: Manatsu no Yoru no Yume,2365,8.04
11158,xxxHOLiC Rou,1513,8.32
11159,xxxHOLiC Shunmuki,1974,8.17


In [34]:
popular_df=popular_df[popular_df['num_rating']>=250].sort_values('avg_rating',ascending=False).head(100)

In [35]:
popular_df

,name,num_rating,avg_rating
4930,Kimi no Na wa.,2199,9.37
2589,Fullmetal Alchemist: Brotherhood,24574,9.26
3048,Gintama°,1386,9.25
9326,Steins;Gate,19283,9.17
3039,Gintama&#039;,3673,9.16
...,...,...,...
7913,Psycho-Pass,14008,8.50
8395,Samurai Champloo,14068,8.50
8939,Shokugeki no Souma: Ni no Sara,3578,8.50
4727,Katanagatari,5317,8.49


In [36]:
popular_df.merge(anime,on='name').drop_duplicates('name')[['name','genre','type','episodes','anime_id','num_rating','avg_rating']]

,name,genre,type,episodes,anime_id,num_rating,avg_rating
0,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,32281,2199,9.37
1,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,5114,24574,9.26
2,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,28977,1386,9.25
3,Steins;Gate,"Sci-Fi, Thriller",TV,24,9253,19283,9.17
4,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9969,3673,9.16
...,...,...,...,...,...,...,...
95,Psycho-Pass,"Action, Police, Psychological, Sci-Fi",TV,22,13601,14008,8.50
96,Samurai Champloo,"Action, Adventure, Comedy, Historical, Samurai...",TV,26,205,14068,8.50
97,Shokugeki no Souma: Ni no Sara,"Ecchi, School, Shounen",TV,13,32282,3578,8.50
98,Katanagatari,"Action, Adventure, Historical, Martial Arts, R...",TV,12,6594,5317,8.49


In [37]:
rating_with_name


,user_id,anime_id,rating_x,name,genre,type,episodes,rating_y,members
0,1,20,-1,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297
1,1,24,-1,School Rumble,"Comedy, Romance, School, Shounen",TV,26,8.06,178553
2,1,79,-1,Shuffle!,"Comedy, Drama, Ecchi, Fantasy, Harem, Magic, R...",TV,24,7.31,158772
3,1,226,-1,Elfen Lied,"Action, Drama, Horror, Psychological, Romance,...",TV,13,7.85,623511
4,1,241,-1,Girls Bravo: First Season,"Comedy, Ecchi, Fantasy, Harem, Romance, School",TV,11,6.69,84395
...,...,...,...,...,...,...,...,...,...
7813605,73515,16512,7,Devil Survivor 2 The Animation,"Action, Demons, Supernatural",TV,13,7.06,101266
7813606,73515,17187,9,Ghost in the Shell: Arise - Border:1 Ghost Pain,"Mecha, Police, Psychological, Sci-Fi",Movie,1,7.64,31747
7813607,73515,22145,10,Kuroshitsuji: Book of Circus,"Comedy, Demons, Fantasy, Historical, Shounen, ...",TV,10,8.37,122895
7813608,73516,790,9,Ergo Proxy,"Mystery, Psychological, Sci-Fi",TV,23,8.03,265005


In [38]:
rating_with_name.groupby('user_id').count()['rating_y']

user_id
1        153
2          3
3         94
4         52
5        467
        ... 
73512     13
73513     33
73514      1
73515    196
73516      2
Name: rating_y, Length: 73515, dtype: int64

In [39]:
result = (rating_with_name.groupby('user_id').count()['rating_y'] > 200 ).value_counts()
print(result)


rating_y
False    62322
True     11193
Name: count, dtype: int64


In [40]:
x = (rating_with_name.groupby('user_id').count()['rating_y'] > 200)
padhe_likhe_users=x[x].index

In [41]:
# Check if `rating_with_name` is a DataFrame with a `user_id` column
if isinstance(rating_with_name, pd.DataFrame) and 'user_id' in rating_with_name.columns:
    # Apply the filter for `padhe_likhe_users`
    filtered_rating_with_name = rating_with_name[rating_with_name['user_id'].isin(padhe_likhe_users)]

    # Proceed with further operations
    x = filtered_rating_with_name.groupby('user_id').count()['rating_y'] > 200
    padhe_likhe_users = x[x].index
else:
    print("rating_with_name is not a DataFrame or 'user_id' column is missing.")


In [42]:
y=filtered_rating_with_name.groupby('name').count()['rating_x']>=50
famous_anime=y[y].index

In [43]:
# Ensure that `filtered_rating_with_name` is a DataFrame and contains a 'name' column
if isinstance(filtered_rating_with_name, pd.DataFrame) and 'name' in filtered_rating_with_name.columns:
    # Group by anime 'name' and count occurrences in 'rating_x', then filter where count >= 50
    y = filtered_rating_with_name.groupby('name').count()['rating_x'] >=50
    
    # Get the list of 'name' indices that meet the threshold
    famous_anime_names = y[y].index
    
    # Check if famous_anime_names contains the expected values
    print("famous_anime_names:", famous_anime_names)

    # Ensure `famous_anime` is structured correctly and contains a 'name' column
    if isinstance(famous_anime, pd.DataFrame) and 'name' in famous_anime.columns:
        # Filter famous_anime to only include names in famous_anime_names if it's non-empty
        if len(famous_anime_names) > 0:
            filtered_famous_anime = famous_anime[famous_anime['name'].isin(famous_anime_names)]
            print("Filtered famous_anime:", filtered_famous_anime)
        else:
            print("No anime found with the specified count threshold.")
    else:
        print("famous_anime is not a DataFrame or 'name' column is missing.")
else:
    print("filtered_rating_with_name is not a DataFrame or 'name' column is missing.")


famous_anime_names: Index(['&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hatsukoi',
       '&quot;Bungaku Shoujo&quot; Memoire',
       '&quot;Bungaku Shoujo&quot; Movie', '.hack//G.U. Returner',
       '.hack//G.U. Trilogy', '.hack//G.U. Trilogy: Parody Mode',
       '.hack//Gift', '.hack//Intermezzo', '.hack//Liminality',
       '.hack//Quantum',
       ...
       'ef: A Tale of Memories.', 'ef: A Tale of Memories. - Prologue',
       'ef: A Tale of Memories. - Recollections', 'iDOLM@STER Xenoglossia',
       's.CRY.ed', 'xxxHOLiC', 'xxxHOLiC Kei',
       'xxxHOLiC Movie: Manatsu no Yoru no Yume', 'xxxHOLiC Rou',
       'xxxHOLiC Shunmuki'],
      dtype='object', name='name', length=5307)
famous_anime is not a DataFrame or 'name' column is missing.


In [44]:
famous_anime

Index(['&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hatsukoi',
       '&quot;Bungaku Shoujo&quot; Memoire',
       '&quot;Bungaku Shoujo&quot; Movie', '.hack//G.U. Returner',
       '.hack//G.U. Trilogy', '.hack//G.U. Trilogy: Parody Mode',
       '.hack//Gift', '.hack//Intermezzo', '.hack//Liminality',
       '.hack//Quantum',
       ...
       'ef: A Tale of Memories.', 'ef: A Tale of Memories. - Prologue',
       'ef: A Tale of Memories. - Recollections', 'iDOLM@STER Xenoglossia',
       's.CRY.ed', 'xxxHOLiC', 'xxxHOLiC Kei',
       'xxxHOLiC Movie: Manatsu no Yoru no Yume', 'xxxHOLiC Rou',
       'xxxHOLiC Shunmuki'],
      dtype='object', name='name', length=5307)

In [45]:
famous_anime_names

Index(['&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hatsukoi',
       '&quot;Bungaku Shoujo&quot; Memoire',
       '&quot;Bungaku Shoujo&quot; Movie', '.hack//G.U. Returner',
       '.hack//G.U. Trilogy', '.hack//G.U. Trilogy: Parody Mode',
       '.hack//Gift', '.hack//Intermezzo', '.hack//Liminality',
       '.hack//Quantum',
       ...
       'ef: A Tale of Memories.', 'ef: A Tale of Memories. - Prologue',
       'ef: A Tale of Memories. - Recollections', 'iDOLM@STER Xenoglossia',
       's.CRY.ed', 'xxxHOLiC', 'xxxHOLiC Kei',
       'xxxHOLiC Movie: Manatsu no Yoru no Yume', 'xxxHOLiC Rou',
       'xxxHOLiC Shunmuki'],
      dtype='object', name='name', length=5307)

In [46]:
final_rating=filtered_rating_with_name[filtered_rating_with_name['name'].isin(famous_anime)]

In [47]:
final_rating

,user_id,anime_id,rating_x,name,genre,type,episodes,rating_y,members
302,5,6,8,Trigun,"Action, Comedy, Sci-Fi",TV,26,8.32,283069
303,5,15,6,Eyeshield 21,"Action, Comedy, Shounen, Sports",TV,145,8.08,83648
304,5,17,6,Hungry Heart: Wild Striker,"Comedy, Shounen, Slice of Life, Sports",TV,52,7.74,13469
305,5,18,6,Initial D Fourth Stage,"Action, Cars, Drama, Seinen, Sports",TV,24,8.24,41584
306,5,20,6,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297
...,...,...,...,...,...,...,...,...,...
7813203,73507,8231,5,Princess Lover!: Magical Knight Maria-chan,"Magic, Parody",Special,6,5.87,5960
7813204,73507,8348,5,Tengen Toppa Gurren Lagann: Parallel Works 2,"Mecha, Music",Music,7,7.09,13361
7813205,73507,8440,7,Black Lagoon Omake,Comedy,Special,7,7.02,33923
7813206,73507,8769,8,Ore no Imouto ga Konnani Kawaii Wake ga Nai,"Comedy, Seinen, Slice of Life",TV,12,7.49,321477


In [48]:
final_rating.drop_duplicates()

,user_id,anime_id,rating_x,name,genre,type,episodes,rating_y,members
302,5,6,8,Trigun,"Action, Comedy, Sci-Fi",TV,26,8.32,283069
303,5,15,6,Eyeshield 21,"Action, Comedy, Shounen, Sports",TV,145,8.08,83648
304,5,17,6,Hungry Heart: Wild Striker,"Comedy, Shounen, Slice of Life, Sports",TV,52,7.74,13469
305,5,18,6,Initial D Fourth Stage,"Action, Cars, Drama, Seinen, Sports",TV,24,8.24,41584
306,5,20,6,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297
...,...,...,...,...,...,...,...,...,...
7813203,73507,8231,5,Princess Lover!: Magical Knight Maria-chan,"Magic, Parody",Special,6,5.87,5960
7813204,73507,8348,5,Tengen Toppa Gurren Lagann: Parallel Works 2,"Mecha, Music",Music,7,7.09,13361
7813205,73507,8440,7,Black Lagoon Omake,Comedy,Special,7,7.02,33923
7813206,73507,8769,8,Ore no Imouto ga Konnani Kawaii Wake ga Nai,"Comedy, Seinen, Slice of Life",TV,12,7.49,321477


In [49]:
pt=final_rating.pivot_table(index='name',columns='user_id',values='rating_x')


In [50]:
pt.fillna(0,inplace=True)

In [51]:
pt

user_id,5,7,17,38,43,46,54,123,129,139,...,73460,73462,73476,73477,73485,73491,73499,73502,73503,73507
name,,,,,,,,,,,,,,,,,,,,,
&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hatsukoi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0
&quot;Bungaku Shoujo&quot; Memoire,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
&quot;Bungaku Shoujo&quot; Movie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0
.hack//G.U. Returner,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
.hack//G.U. Trilogy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
xxxHOLiC,2.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,...,0.0,7.0,0.0,0.0,7.0,0.0,0.0,10.0,8.0,10.0
xxxHOLiC Kei,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,8.0,0.0,0.0,10.0,8.0,9.0
xxxHOLiC Movie: Manatsu no Yoru no Yume,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,8.0,0.0,0.0,8.0,0.0,0.0,10.0,0.0,9.0


In [52]:
from sklearn.metrics.pairwise import cosine_similarity

In [53]:
similarity_score=cosine_similarity(pt)

In [54]:
similarity_score.shape

(5307, 5307)

In [55]:
def recommend(anime_name):
    # index fetch
    index = np.where(pt.index == anime_name)[0][0]
    similar_items = sorted(list(enumerate(similarity_score[index])), key=lambda x: x[1], reverse=True)[1:6]
    
    suggestions = []  # Initialize the list to store suggestions
    for i in similar_items:
        print(pt.index[i[0]])
        suggestions.append(i[0])  # Add each suggestion to the list
        
    return suggestions  # Ensure this line is inside the function


In [56]:
recommend('Bleach')

Naruto
Ao no Exorcist
Bleach Movie 4: Jigoku-hen
Fairy Tail
Bleach Movie 3: Fade to Black - Kimi no Na wo Yobu


[3217, 243, 516, 1218, 515]

In [57]:
#from fuzzywuzzy import process  # Fuzzy matching library
#import numpy as np

#def recommend(anime_name):
    # Find the closest match to the input anime_name
    #closest_match = process.extractOne(anime_name, pt.index)
    
    #if closest_match is not None:
        #matched_name = closest_match[0]
        #print(f"Did you mean: {matched_name}?")
        
        # Fetch index of the closest matching anime
       # index = np.where(pt.index == matched_name)[0][0]
       
       # similar_items = sorted(list(enumerate(similarity_score[index])), key=lambda x: x[1], reverse=True)[1:6]
        
        #  suggestions = []  # Initialize the list to store suggestions
        #for i in similar_items:
         #   print(pt.index[i[0]])  # Print recommended anime name
          #  suggestions.append(pt.index[i[0]])  # Add each suggestion to the list
        
       # return suggestions  # Return the list of suggested anime
    #else:
     #   print("No similar anime found. Please try a different title.")
      #  return []

# Example usage:
# recommendations = recommend("Naruto") 


In [58]:
recommend("Naruto")

Death Note
Code Geass: Hangyaku no Lelouch
Sword Art Online
Bleach
Fullmetal Alchemist


[932, 792, 4536, 512, 1317]

In [59]:
import pandas as pd
import numpy as np

# Example of saving a DataFrame `pt`
pt.to_csv('anime_dataset.csv')  # Save DataFrame as CSV

# Saving `similarity_score` as a .npy file
np.save('similarity_score.npy', similarity_score)  # Save NumPy array


In [60]:
popular_df
import pandas as pd
import numpy as np

# Example of saving a DataFrame `pt`
popular_df.to_csv('popularanime_dataset.csv')  # Save DataFrame as CSV

# Saving `similarity_score` as a .npy file
np.save('similarity_score.npy', similarity_score)  # Save NumPy array